In [ ]:
import os
import re
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

def extract_legend(file_name):
    match = re.search(r'B(.*?)_', file_name)
    if match:
        return match.group(1)
    else:
        return "Unknown"

def plot_data(directory, model, ylim):
    # files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    files = [f"{directory}/B16_CN0.0001_LR1e-05.txt", f"{directory}/B32_CN0.0001_LR1e-05.txt", f"{directory}/B64_CN0.0001_LR1e-05.txt", f"{directory}/B128_CN0.0001_LR1e-05.txt", f"{directory}/B256_CN0.0001_LR1e-05.txt"]
    legends = {}
    for file_name in files:
        legend = extract_legend(file_name)
        legends[file_name] = "Batch = " + legend

    for file_path, legend in legends.items():
        with open(file_path, 'r') as file:
            data = file.read()

        episodes = []
        rewards = []
        total_steps = []
        for line in data.strip().split('\n'):
            record = json.loads(line)
            episodes.append(record['episode'])
            rewards.append(record['reward'])
            total_steps.append(record['total_steps'])

        window_size = 50
        reward_series = pd.Series(rewards)
        # rolling_steps = np.convolve(total_steps, np.ones(rolling_window)/rolling_window, mode='valid')
        rolling_rewards = reward_series.rolling(window=window_size, min_periods=1).mean()
        sns.set_style("whitegrid")
        sns.set_palette("Set1")  # Example of using a different color palette
    
    # Plotting with Seaborn
        sns.lineplot(x=total_steps, y=rolling_rewards, linewidth=2, label=legend)  # Customizing line width and adding markers
        sns.set_context("paper") 
       

    plt.xlabel('Total Steps',fontdict={'fontsize': 9, 'fontweight': 'bold'})
    plt.ylabel('Reward (Rolling Average)',fontdict={'fontsize': 9, 'fontweight': 'bold'})
    plt.title(f'{model}: Rolling Average Reward per Episode (Window = {window_size})',fontdict={'fontsize': 11, 'fontweight': 'bold'}, pad=8)
    plt.suptitle("Performance by different batch size", fontstyle="italic")
    plt.legend(loc='upper left')
    plt.xlim(0, 102000)  # Set x-axis range from 0 to 100,000
    plt.ylim(0, ylim)
    plt.tight_layout()
    plt.show()

# REP by Clip norm
plot_data("./logs/random_replay/batch", "DDDQN Random EP", 300)
plot_data("./logs/prioritised_replay/batch", "DDDQN PEP", 500)